In [1]:
import csv
import datetime
import itertools
import os
import re
import requests
import sys
import time
from copy import deepcopy
from collections import namedtuple, defaultdict

from bs4 import BeautifulSoup as BS
from tenacity import retry, stop_after_attempt
import simplejson as json
from selenium import webdriver
from opencc import OpenCC

In [2]:
class WordsCrawler(object):
    
    def __init__(self, output_dir):
        
        self.session = requests.Session()
        self.headers = {
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6",
            "Connection": "keep-alive",
            "Referer": "https://www.google.com.tw/",
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) "\
                          "Chrome/69.0.3497.92 Safari/537.36"
        }
        self.scroll_pause_time = 0.3
        self.driver = webdriver.PhantomJS(executable_path='../phantomjs-2.1.1-linux-x86_64/bin/phantomjs')
        self.output_dir = output_dir
    
    @retry(stop=stop_after_attempt(3))
    def get_bsObj(self, url):
        
        req = self.session.get(url, headers=self.headers)
        if req.url != url:
            return None
        bsObj = BS(req.text, "html.parser")
        return bsObj
    
    def url_generator(self):
        for i in range(1, 242):
            yield "http://bs.chinese-linguipedia.org/api/web/diff/search?category=&page=%s&word=&type=" % i
            
    def crawl_and_save(self):
        
        cc = OpenCC('s2t')
        os.makedirs(self.output_dir, exist_ok=True)
        
        with open(os.path.join(self.output_dir, 'words.csv'), 'w+', encoding='utf-8') as csvfile:
            
            writer = csv.writer(csvfile)
            writer.writerow(['type', 'category', 'tw_word', 'cn_word'])
        
            for url in self.url_generator():
                res = words_crawler.session.get(url, headers=words_crawler.headers)
                word_pairs = json.loads(res.text)['data']['data']
                for word_pair in word_pairs:
                    writer.writerow([word_pair['type'], word_pair['category'], word_pair['tw_word'], cc.convert(word_pair['cn_word'])])

In [3]:
words_crawler = WordsCrawler('words')
words_crawler.crawl_and_save()

/home/mercedes/VirtualEnvs/SDML/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
